In [2]:
import pandas as pd
from pulp import LpProblem, LpMaximize, LpVariable, lpSum, LpBinary, LpStatus, value, PULP_CBC_CMD

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/mk/tvz_pqr97_778k6rbbp07ylm0000gn/T/dae2d859688646b4a191f48b2d66f46f-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/mk/tvz_pqr97_778k6rbbp07ylm0000gn/T/dae2d859688646b4a191f48b2d66f46f-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 38195 COLUMNS
At line 118381 RHS
At line 156572 BOUNDS
At line 158443 ENDATA
Problem MODEL has 38190 rows, 5610 columns and 76360 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 55 - 0.02 seconds
Cgl0004I processed model has 1650 rows, 85 columns (85 integer (85 of which binary)) and 3300 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 0 integers unsatisfied sum 

In [3]:
blocks = pd.read_csv("/Users/muskansingh/OR-MILP/or-milp-env/data-folder/blocks.csv")
loops = pd.read_csv("/Users/muskansingh/OR-MILP/or-milp-env/data-folder/loops.csv")
stations = pd.read_csv("/Users/muskansingh/OR-MILP/or-milp-env/data-folder/stations.csv")
passenger_trains = pd.read_csv("/Users/muskansingh/OR-MILP/or-milp-env/data-folder/scheduled_trains.csv")
goods_trains = pd.read_csv("/Users/muskansingh/OR-MILP/or-milp-env/data-folder/unscheduled_trains.csv")

In [4]:
# Merge passenger and goods trains
passenger_trains = passenger_trains.reset_index(drop=True)
passenger_trains['train_id'] = passenger_trains.index

goods_trains = goods_trains.reset_index(drop=True)
goods_trains['train_id'] = goods_trains.index + len(passenger_trains)

trains = pd.concat([passenger_trains, goods_trains], ignore_index=True)
trains['train_id'] = trains.index

In [5]:
# Naive scheduling
def naive_schedule(trains, blocks):
    completed = []
    for _, t_row in trains.iterrows():
        # Start at allowed time if available, else 0
        start_time = t_row.get("start_time_min_allowed", 0)
        feasible = True
        for _, b_row in blocks.iterrows():
            block_length = b_row.end_km - b_row.start_km
            speed = min(t_row.get('max_speed_kmph', 110), b_row.block_velocity_kmph)
            travel_time = block_length / speed * 60  # in minutes
            start_time += travel_time
            # Naive: ignore conflicts
        if feasible:
            completed.append(t_row.train_id)
    return completed

In [8]:
# MILP scheduling
def milp_schedule(trains, blocks, loops):
    model = LpProblem("Maximize_Throughput", LpMaximize)
    e, l, x = {}, {}, {}
    
    # Decision variables
    for _, t_row in trains.iterrows():
        for _, b_row in blocks.iterrows():
            key = (t_row.train_id, b_row.block_id)
            e[key] = LpVariable(f"e_{t_row.train_id}_{b_row.block_id}", lowBound=0)
            l[key] = LpVariable(f"l_{t_row.train_id}_{b_row.block_id}", lowBound=0)
            x[key] = LpVariable(f"x_{t_row.train_id}_{b_row.block_id}", cat=LpBinary)
        for _, l_row in loops.iterrows():
            key = (t_row.train_id, l_row.loop_id)
            e[key] = LpVariable(f"e_{t_row.train_id}_{l_row.loop_id}", lowBound=0)
            l[key] = LpVariable(f"l_{t_row.train_id}_{l_row.loop_id}", lowBound=0)
            x[key] = LpVariable(f"x_{t_row.train_id}_{l_row.loop_id}", cat=LpBinary)

    # Objective: maximize trains completing last block
    model += lpSum([x[t_row.train_id, blocks.iloc[-1].block_id] for _, t_row in trains.iterrows()]), "Maximize_Throughput"
    # Travel time constraints
    for _, t_row in trains.iterrows():
        for _, b_row in blocks.iterrows():
            block_length = b_row.end_km - b_row.start_km
            speed = min(t_row.get('max_speed_kmph', 110), b_row.block_velocity_kmph)
            travel_time = block_length / speed * 60
            model += l[t_row.train_id, b_row.block_id] >= e[t_row.train_id, b_row.block_id] + travel_time, \
                     f"TravelTime_{t_row.train_id}_{b_row.block_id}"

    # Goods train start time
    for _, t_row in goods_trains.iterrows():
        first_block = blocks.iloc[0]['block_id']
        model += e[t_row['train_id'], first_block] >= t_row['start_time_min_allowed'], f"GoodsStart_{t_row['train_id']}"

    # Direction conflict
    up_trains = trains[trains.direction=='up']
    down_trains = trains[trains.direction=='down']
    for _, b_row in blocks.iterrows():
        for _, t_up in up_trains.iterrows():
            for _, t_down in down_trains.iterrows():
                model += x[t_up.train_id, b_row.block_id] + x[t_down.train_id, b_row.block_id] <= 1, \
                         f"Direction_{t_up.train_id}_{t_down.train_id}_{b_row.block_id}"
    # Solve MILP
    solver = PULP_CBC_CMD(msg=1)
    model.solve(solver)

    completed_trains = []
    for _, t_row in trains.iterrows():
        last_block = blocks.iloc[-1]['block_id']
        if value(x[t_row.train_id, last_block]) > 0.5:
            completed_trains.append(t_row.train_id)
    return completed_trains


In [9]:
naive = naive_schedule(trains, blocks)
milp = milp_schedule(trains, blocks, loops)

print("Naive trains completed:", len(naive))
print("MILP trains completed:", len(milp))
print("Improvement:", len(milp) - len(naive))

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/mk/tvz_pqr97_778k6rbbp07ylm0000gn/T/803756c2749949729d2bd4cd1a5913cb-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/mk/tvz_pqr97_778k6rbbp07ylm0000gn/T/803756c2749949729d2bd4cd1a5913cb-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 38195 COLUMNS
At line 118381 RHS
At line 156572 BOUNDS
At line 158443 ENDATA
Problem MODEL has 38190 rows, 5610 columns and 76360 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 55 - 0.03 seconds
Cgl0004I processed model has 1650 rows, 85 columns (85 integer (85 of which binary)) and 3300 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 0 integers unsatisfied sum 